# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [16]:
# AzureML imports
from azureml.core import Workspace
from azureml.core import Dataset
from azureml.core import Datastore
from azureml.core import Experiment
from azureml.core import Environment
from azureml.core import Model
from azureml.core.compute import ComputeTarget
from azureml.widgets import RunDetails
# Hyperdrive imports
from azureml.core import ScriptRunConfig
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
# Other imports
import numpy as np
import pandas as pd
import seaborn as sns
import joblib

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
# Data is managed in the train.py script
# The data downloaded in the train

In [2]:
ws = Workspace.from_config()

In [ ]:
# Import dataset from datastore and convert into dataframe for exploration
dataset = Dataset.get_by_name(ws, name='pima')
df = dataset.to_pandas_dataframe()

In [ ]:
experiment_name = 'pima-exp-hyperdrive'
experiment=Experiment(ws, experiment_name)

In [3]:
%%writefile conda_dependencies.yml

dependencies:
- python=3.6.2
- scikit-learn
- pandas
- pip:
    - azureml-defaults

Writing conda_dependencies.yml


In [4]:
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='./conda_dependencies.yml')

In [6]:
# Choose a name for your CPU cluster
cpu_cluster_name = "cpucluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # To use a different region for the compute, add a location='<region>' parameter
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [7]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling( {
        '--C': uniform(0.5,1.5),
        '--max_iter': choice(100,150,200,250)
    }
)

#TODO: Create your estimator and hyperdrive config
estimator_config = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      compute_target=cpu_cluster,
                      environment=sklearn_env)

hyperdrive_run_config = HyperDriveConfig(run_config=estimator_config,
                             hyperparameter_sampling=param_sampling,
                             policy=early_termination_policy,
                             primary_metric_name="Accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=10,
                             max_concurrent_runs=4)

In [10]:
#TODO: Submit your experiment
run = experiment.submit(hyperdrive_run_config)
run.wait_for_completion(show_output=True)

RunId: HD_53594929-6104-4c36-a34f-d16b535f4bf8
Web View: https://ml.azure.com/runs/HD_53594929-6104-4c36-a34f-d16b535f4bf8?wsid=/subscriptions/5a4ab2ba-6c51-4805-8155-58759ad589d8/resourcegroups/aml-quickstarts-170218/workspaces/quick-starts-ws-170218&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-12-29T19:04:43.583790][API][INFO]Experiment created<END>\n""<START>[2021-12-29T19:04:44.654862][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-12-29T19:04:45.385950][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_53594929-6104-4c36-a34f-d16b535f4bf8
Web View: https://ml.azure.com/runs/HD_53594929-6104-4c36-a34f-d16b535f4bf8?wsid=/subscriptions/5a4ab2ba-6c51-4805-8155-58759ad589d8/resourcegroups/aml-quickstarts-170218/workspaces/quick-starts-ws-170218&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



{'runId': 'HD_53594929-6104-4c36-a34f-d16b535f4bf8',
 'target': 'cpucluster',
 'status': 'Completed',
 'startTimeUtc': '2021-12-29T19:04:43.280738Z',
 'endTimeUtc': '2021-12-29T19:11:26.639927Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'db7ddcad-9b8d-4b08-8600-4f09c1b144f9',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1063-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.36.0',
  'space_size': 'infinite_space_size',
  'score': '0.7727272727272727',
  'best_child_run_id': 'HD_53594929-6104-4c36-a34f-d16b535f4bf8_9',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg170218.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_53594929-6104-4c36-a34f-d16b53

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [11]:
RunDetails(run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [12]:
best_run = run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])

Best Run Id:  HD_53594929-6104-4c36-a34f-d16b535f4bf8_9

 Accuracy: 0.7727272727272727


In [13]:
parameter_values

['--C', '1.3402538583878547', '--max_iter', '100']

In [14]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
pima-exp-hyperdrive,HD_53594929-6104-4c36-a34f-d16b535f4bf8_9,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [15]:
best_run.get_file_names()

['logs/azureml/19_azureml.log',
 'logs/azureml/dataprep/backgroundProcess.log',
 'logs/azureml/dataprep/backgroundProcess_Telemetry.log',
 'logs/azureml/dataprep/rslex.log',
 'outputs/model.joblib',
 'system_logs/cs_capability/cs-capability.log',
 'system_logs/hosttools_capability/hosttools-capability.log',
 'system_logs/lifecycler/execution-wrapper.log',
 'system_logs/lifecycler/lifecycler.log',
 'system_logs/lifecycler/vm-bootstrapper.log',
 'user_logs/std_log.txt']

In [18]:
#TODO: Save the best model

In [17]:
model=best_run.register_model(model_name='best_model', model_path='outputs/model.joblib')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

